<a href="https://colab.research.google.com/github/adhang/data-science-digitalskola/blob/update/08.%20Advanced%20Pandas%20Dataframe/Homework%20-%20Advanced%20Pandas%20Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework - Advanced Dataframe
Author: Adhang Muntaha Muhammad

[![LinkedIn](https://img.shields.io/badge/linkedin-0077B5?style=for-the-badge&logo=linkedin&logoColor=white&link=https://www.linkedin.com/in/adhangmuntaha/)](https://www.linkedin.com/in/adhangmuntaha/)
[![GitHub](https://img.shields.io/badge/github-121011?style=for-the-badge&logo=github&logoColor=white&link=https://github.com/adhang)](https://github.com/adhang)
[![Kaggle](https://img.shields.io/badge/kaggle-20BEFF?style=for-the-badge&logo=kaggle&logoColor=white&link=https://www.kaggle.com/adhang)](https://www.kaggle.com/adhang)
[![Tableau](https://img.shields.io/badge/tableau-E97627?style=for-the-badge&logo=tableau&logoColor=white&link=https://public.tableau.com/app/profile/adhang)](https://public.tableau.com/app/profile/adhang)
___

## Problems and Tasks

**Dataset**
1. `insurance.csv`

**Tasks**
1. Create a new column `BMI_category` using a lambda function, where the values are:
  - `High BMI` - if `bmi` >= 50
  - `Medium BMI` - if `bmi` >= 30 and `bmi` < 50
  - `Low BMI` - if `bmi` < 30
2. Create a pivot table to get the `min`, `max`, and `mean` of the `bmi` column, grouped by the `BMI_category` column.
3. Using lambda function, change the datapoints on the `sex`, `smoker`, `region`, and `BMI_category` columns to upper-case.

## Solutions

### Importing Libraries

In [ ]:
import pandas as pd

### Reading Dataset

In [ ]:
file_path = 'https://raw.githubusercontent.com/densaiko/data_science_learning/main/dataset/insurance.csv'

data = pd.read_csv(file_path)
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Solution - Task 1
Before we create a new column, let's take a look at the data type of each column. Just to make sure when we do some numerical calculations or condition checking, it won't raise an error.

In [ ]:
data.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

The `age`, `bmi`, `children`, and `charges` has numeric data type. So we can do some calculations.

My lambda function is similar to this pseudecde
```
if x < 30:
  'Low BMI'
elif x < 50:
  'Medium BMI'
else:
  'High BMI'
```

On the second condition, I'm not using `x >= 30 and x < 50` because the second condition is evaluated only when the first condition is not satisfied. The first condition is `x < 30`, so, if `x` didn't meet the first condition means `x` is always larger or equal to 30 (`x >= 30`).

In [ ]:
data['BMI_category'] = data['bmi'].apply(lambda x: 'Low BMI' if x < 30 else ('Medium BMI' if x < 50 else 'High BMI'))
data.head()

,age,sex,bmi,children,smoker,region,charges,BMI_category
0,19,female,27.900,0,yes,southwest,16884.92400,Low BMI
1,18,male,33.770,1,no,southeast,1725.55230,Medium BMI
2,28,male,33.000,3,no,southeast,4449.46200,Medium BMI
3,33,male,22.705,0,no,northwest,21984.47061,Low BMI
4,32,male,28.880,0,no,northwest,3866.85520,Low BMI


### Solution - Task 2
Create a pivot table using the `BMI_category` colum as the index.

In [ ]:
pd.pivot_table(data, values='bmi', index='BMI_category', aggfunc=['min', 'max', 'mean']).round(2)

,min,max,mean
,bmi,bmi,bmi
BMI_category,,,
High BMI,50.38,53.13,52.03
Low BMI,15.96,29.92,25.51
Medium BMI,30.00,49.06,35.19


Can we swap the `bmi` column name with the aggregation names? Like this:

|                  | **bmi** |         |          |
|:----------------:|:-------:|:-------:|:--------:|
|                  | **min** | **max** | **mean** |
| **BMI_category** |         |         |          |
|   **High BMI**   |  50.38  |  53.13  |   52.03  |
|    **Low BMI**   |  15.96  |  29.92  |   25.51  |
|  **Medium BMI**  |  30.00  |  49.06  |   35.19  |

To do that, instead of passing a list to the `aggfunc` parameter, we can pass a dictionary.

In [ ]:
pd.pivot_table(data, values='bmi', index='BMI_category', aggfunc={'bmi':['min', 'max', 'mean']}).round(2)

,max,mean,min
BMI_category,,,
High BMI,53.13,52.03,50.38
Low BMI,29.92,25.51,15.96
Medium BMI,49.06,35.19,30.00


Whops, where's the `bmi` column name?

It's hidden because we only pass a single column to the `values` parameter. Let's try with 2 columns, the `bmi` and `charges` columns.

In [ ]:
pd.pivot_table(data, values=['bmi', 'charges'], index='BMI_category', aggfunc={'bmi':['min', 'max'], 'charges':['min','max']}).round(2)

bmi          charges         
                max    min       max      min
BMI_category                                 
High BMI      53.13  50.38  44501.40  1163.46
Low BMI       29.92  15.96  38245.59  1121.87
Medium BMI    49.06  30.00  63770.43  1131.51

See? We have the column names above the aggregation names.
<br><br>
Another way to do this is using `swaplevel()` method. In my opinion, I think this method is better than the previous method, especially if we only pass a single column to the `values` parameter.
<br><br>
Syntax
```
data.swaplevel(i, j, axis=0)
```
This method is used to swap level between `i` and `j`. The default axis is 0, which means it's used to swap the index (on multi-index). If we want to swap multi-column, set the axis to 1.
<br><br>
In this case, the aggregation functions (`min` and `max`) are on the first level (level 0), and the column name (`charges`) is on the second level (level 1). So, we will do a swap between level 0 and level 1 on axis 1 (column axis).

In [ ]:
data_swap = pd.pivot_table(data, values='bmi', index='BMI_category', aggfunc=['min', 'max', 'mean']).round(2)
data_swap = data_swap.swaplevel(0, 1, axis=1)
data_swap

bmi              
                min    max   mean
BMI_category                     
High BMI      50.38  53.13  52.03
Low BMI       15.96  29.92  25.51
Medium BMI    30.00  49.06  35.19

Alternatively, we can use `swaplevel()` method on the `columns` attribute.

In [ ]:
data_swap = pd.pivot_table(data, values='bmi', index='BMI_category', aggfunc=['min', 'max', 'mean']).round(2)
data_swap.columns = data_swap.columns.swaplevel(0,1)
data_swap

bmi              
                min    max   mean
BMI_category                     
High BMI      50.38  53.13  52.03
Low BMI       15.96  29.92  25.51
Medium BMI    30.00  49.06  35.19

### Solution - Task 3
Change all string values to upper-case using a lambda function.

In [ ]:
column_list = ['sex','smoker','region','BMI_category']

for i in column_list:
  data[i] = data[i].apply(lambda x: x.upper())
  
data.head()

,age,sex,bmi,children,smoker,region,charges,BMI_category
0,19,FEMALE,27.900,0,YES,SOUTHWEST,16884.92400,LOW BMI
1,18,MALE,33.770,1,NO,SOUTHEAST,1725.55230,MEDIUM BMI
2,28,MALE,33.000,3,NO,SOUTHEAST,4449.46200,MEDIUM BMI
3,33,MALE,22.705,0,NO,NORTHWEST,21984.47061,LOW BMI
4,32,MALE,28.880,0,NO,NORTHWEST,3866.85520,LOW BMI
